In [2]:
import stim
from pathlib import Path

def stim_to_qasm(file_path: Path) -> str:
    with file_path.open("r") as f:
        circuit_text = f.read()

    circuit = stim.Circuit(circuit_text)
    clean_circuit = circuit.without_noise()
    qasm_str = clean_circuit.to_qasm(open_qasm_version=2, skip_dets_and_obs=True)
    return qasm_str

def clean_stim(file_path: Path) -> str:
    with file_path.open("r") as f:
        circuit_text = f.read()

    circuit = stim.Circuit(circuit_text)
    clean_circuit = stim.Circuit()

    for instruction in circuit:
        name = instruction.name
        if name in {"DETECTOR", "OBSERVABLE_INCLUDE", "QUBIT_COORDS"}:
            continue  # skip metadata and detectors
        if any("rec[" in str(t) for t in instruction.targets_copy()):
            continue  # skip feedback instructions

        targets = instruction.targets_copy()

        # Special handling for multi-target CX gates
        if name == "CX":
            for i in range(0, len(targets), 2):
                clean_circuit.append(name, [targets[i], targets[i+1]])
        else:
            # For other gates, split them into single-target gates
            for target in targets:
                clean_circuit.append(name, target)

    return str(clean_circuit)


source_folder = Path("/Users/garn195/Project Repositories/Stim_Benchmarks/Circuits/Pauli_product")
qasm_folder = Path("/Users/garn195/Project Repositories/NWQ-Sim/stabilizer/stim_to_qasm_files")
clean_stim_folder = Path("/Users/garn195/Project Repositories/NWQ-Sim/stabilizer/stim_to_stim_files")

qasm_folder.mkdir(parents=True, exist_ok=True)
clean_stim_folder.mkdir(parents=True, exist_ok=True)

stim_files = list(source_folder.rglob("*"))
print(f"Found {len(stim_files)} stim files.")

for stim_file in stim_files:
    try:
        qasm_str = stim_to_qasm(stim_file)
        qasm_output_path = qasm_folder / (stim_file.stem + ".qasm")
        qasm_output_path.write_text(qasm_str)

        clean_stim_str = clean_stim(stim_file)
        clean_stim_output_path = clean_stim_folder / (stim_file.name + ".stim")
        clean_stim_output_path.write_text(clean_stim_str)

        print(f"Converted: {stim_file.name}")

    except Exception as e:
        print(f"Error processing {stim_file.name}: {e}")



Found 297 stim files.
Converted: name=multi_measure_Z_one_round_d=9_space=7_weight=6
Converted: name=multi_measure_Z_one_round_d=9_space=6_weight=6
Converted: name=multi_measure_Z_one_round_d=7_space=10_weight=10
Converted: name=multi_measure_Z_one_round_d=7_space=0_weight=6


KeyboardInterrupt: 